In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

if os.path.exists('./file/5.oem'):
    shutil.rmtree('./file/5.oem')

os.makedirs('./file/5.oem')

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/2.part'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

for file in list_file:
    
    # = = = = = = = = = = = = = = =

    output_correct, output_error = pd.DataFrame(), pd.DataFrame()

    # = = = = = = = = = = = = = = =

    input_ = pd.read_excel(f'./file/2.part/{file}',
                           header=0,
                           dtype=str).fillna('')
    
    input_['No'] = input_['No'].astype(int)

    print(f'总数量：{len(input_)}')
    print()

    # = = = = = = = = = = = = = = =


    work = Queue()
    for a in range(len(input_)):
        work.put_nowait(a)

    # = = = = = = = = = = = = = = =

    def crawler():
        global output_correct, output_error

        while not work.empty():
            a = work.get_nowait()

            # = = = = = = = = = = = = = = =

            crawler_status = 'error'

            # = = = = = = = = = = = = = = =

            try:
                b = 0
                while True:
                    b += 1
                    
                    try:
                        request_url, check_url = f'''https://productdesk-api.cellacore.net/legacy/cardone/FinishedGood/?q=productDeskItemNumber.keyword:{input_.loc[a, 'Part Number']}&size=1&useCache=true&auth={input_.loc[a, 'Auth']}''', input_.loc[a, 'Url']
                        
                        resp = requests.get(request_url,
                                            headers=get_header(),
                                            proxies=get_proxy(),
                                            timeout=(10, 10))

                        if resp.status_code == 200:
                            if resp.json():
                                dict_ = resp.json()[0]
                                break
                    except KeyboardInterrupt:
                        break
                    except:
                        continue

                # = = = = = = = = = = = = = = =

                list_oem, list_dict = [], dict_['CrossReferences'] if 'CrossReferences' in dict_ else []
                for dict_ in list_dict:
                    if dict_['referenceType'].strip() == 'OE Part':
                        list_oem.append(dict_['reference'].strip())
                
                oem = ';'.join(sorted(list(set(list_oem)), key=list_oem.index))

                # = = = = = = = = = = = = = = =

                df_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                                         'Type': input_.loc[a, 'Type'],
                                         'Part Number': input_.loc[a, 'Part Number'],
                                         'Title': input_.loc[a, 'Title'],
                                         'Subtitle': input_.loc[a, 'Subtitle'],
                                         'Application Summary': input_.loc[a, 'Application Summary'],
                                         'OEM': oem,
                                         'Picture': input_.loc[a, 'Picture'],
                                         'Url': input_.loc[a, 'Url'],
                                         'Json_Src': input_.loc[a, 'Json_Src'],
                                         'Json_Info': input_.loc[a, 'Json_Info'],
                                         'Bundle': input_.loc[a, 'Bundle'],
                                         'Auth': input_.loc[a, 'Auth']}])
                
                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'
            except:
                df_temp = pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)
                df_temp.loc[0, 'Request_Url'] = request_url
                df_temp.loc[0, 'Check_Url'] = check_url
                
                output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and work.qsize() == 0 or work.qsize() % 1_000 == 0 and work.qsize() != 0:
                clear_output()

            print(f'''[状态：{crawler_status}，尝试次数：{b}] - {(list_file.index(file) + 1) / len(list_file) * 100:.2f}% - {file.removesuffix('.xlsx')} > {input_.loc[a, 'No']}.{request_url}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

    # = = = = = = = = = = = = = = =

    list_task = []
    for _ in range(5):
        task = gevent.spawn(crawler)
        list_task.append(task)
    gevent.joinall(list_task)

    print('输出ing...')
    print()
    if not output_correct.empty:
        output_correct = output_correct.sort_values(by=['No'],
                                                    ascending=[True],
                                                    ignore_index=True)
        output_correct.to_excel(f'''./file/5.oem/{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    if not output_error.empty:
        output_error = output_error.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
        output_error.to_excel(f'''./file/{file.removesuffix('.xlsx')}-oem_error.xlsx''', index=False)
        print('爬虫存在error')
        print()

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/5.oem'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

from tqdm import tqdm
import re

for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'./file/5.oem/{file}',
              f'''./file/5.oem/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)}''')

print('Done ~')

总数量：10

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 4.https://productdesk-api.cellacore.net/legacy/cardone/FinishedGood/?q=productDeskItemNumber.keyword:40-10006&size=1&useCache=true&auth=mXWIhUuYvk3hklMcdwLi4lwVd3s6VASw
[剩余数量：5] - [当前时间：13:59:34]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 5.https://productdesk-api.cellacore.net/legacy/cardone/FinishedGood/?q=productDeskItemNumber.keyword:40-10007&size=1&useCache=true&auth=mXWIhUuYvk3hklMcdwLi4lwVd3s6VASw
[剩余数量：4] - [当前时间：13:59:34]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 2.https://productdesk-api.cellacore.net/legacy/cardone/FinishedGood/?q=productDeskItemNumber.keyword:40-10000&size=1&useCache=true&auth=mXWIhUuYvk3hklMcdwLi4lwVd3s6VASw
[剩余数量：3] - [当前时间：13:59:34]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 3.https://productdesk-api.cellacore.net/legacy/cardone/FinishedGood/?q=productDeskItemNumber.keyword:40-10005&size=1&useCache=true&auth=mXWIhUuYvk3hklMcdwLi4lwVd3s6VASw
[剩余数量：2] - [当前时间：13:59:35]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 7.https://prod

Progress: 100%|██████████████████████████████| 2/2 [00:00<00:00, 1959.04it/s]

Done ~
